# RIS Auto-Research Engine - Quick Start Guide

This notebook provides a quick introduction to the RIS Auto-Research Engine. You'll learn how to:
- Verify your installation
- Run your first experiments
- Visualize and compare results

**Expected Runtime:** ~2-3 minutes for the quick test campaign (2 experiments).

**Prerequisites:** Make sure you've installed the package with `pip install -e .` from the repository root.

## 1. Installation Check

First, let's verify that all required modules are properly installed and can be imported.

In [ ]:
# Import core modules
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

# Import RIS Engine components
try:
    from ris_research_engine.ui import RISEngine
    from ris_research_engine.foundation import __version__
    from ris_research_engine.plugins.probes import list_probes
    from ris_research_engine.plugins.models import list_models
    print("✓ RIS Research Engine successfully imported!")
    print(f"  Version: {__version__ if '__version__' in dir() else 'development'}")
except ImportError as e:
    print(f"✗ Import error: {e}")
    print("  Please install the package: pip install -e .")
    sys.exit(1)

# Check Python version
print(f"\n✓ Python version: {sys.version.split()[0]}")
print(f"✓ NumPy version: {np.__version__}")
print(f"✓ Pandas version: {pd.__version__}")
print(f"✓ Matplotlib version: {plt.matplotlib.__version__}")

# List available probes and models
print("\nAvailable Probes:")
for probe in list_probes():
    print(f"  - {probe}")

print("\nAvailable Models:")
for model in list_models():
    print(f"  - {model}")

print("\n✓ All components ready!")

## 2. Initialize RIS Engine

Create an instance of the RISEngine with default settings. Results will be stored in `results.db` and outputs saved to `outputs/` directory.

In [ ]:
# Initialize engine
engine = RISEngine(
    db_path="results.db",
    output_dir="outputs"
)

print("RIS Engine initialized successfully!")
print(f"  Database: {engine.db_path}")
print(f"  Output directory: {engine.output_dir}")

## 3. Run Quick Test Campaign

Run the `quick_test.yaml` configuration which includes 2 experiments:
- Hadamard probe (expected best performance)
- Random uniform probe (baseline)

This campaign uses small system parameters (N=16, K=16, M=4) and only 10 epochs for fast validation.

In [ ]:
# Path to quick test configuration
config_path = "../configs/search_spaces/quick_test.yaml"

# Run the campaign
print("Starting quick test campaign...\n")
try:
    campaign_result = engine.search(config_path)
    print("\n✓ Campaign completed successfully!")
except Exception as e:
    print(f"\n✗ Campaign failed: {e}")
    raise

## 4. Display Results Table

View a summary of all experiments in a clean tabular format showing key metrics and configurations.

In [ ]:
# Extract experiment results
experiments = campaign_result['experiments']

# Create summary table
results_data = []
for exp in experiments:
    results_data.append({
        'Experiment': exp['config']['name'],
        'Probe': exp['config']['probe_type'],
        'Model': exp['config']['model_type'],
        'Status': exp['status'],
        'Top-1 Acc': f"{exp['metrics'].get('top_1_accuracy', 0):.3f}",
        'Val Loss': f"{exp['metrics'].get('val_loss', 0):.4f}",
        'Time (s)': f"{exp['training_time_seconds']:.1f}",
        'Epochs': exp['total_epochs']
    })

results_df = pd.DataFrame(results_data)
print("\n" + "="*80)
print("EXPERIMENT RESULTS")
print("="*80)
print(results_df.to_string(index=False))
print("="*80)

## 5. Plot Training Curves

Visualize the training and validation loss curves for both experiments to understand convergence behavior.

In [ ]:
# Set up plot style
plt.style.use('seaborn-v0_8-darkgrid' if 'seaborn-v0_8-darkgrid' in plt.style.available else 'default')

# Create subplots for each experiment
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Training Curves - Quick Test Campaign', fontsize=16, fontweight='bold')

for idx, exp in enumerate(experiments):
    if 'training_history' in exp:
        history = exp['training_history']
        epochs = range(1, len(history['train_loss']) + 1)
        
        # Plot on subplot
        ax = axes[idx]
        ax.plot(epochs, history['train_loss'], 'b-', label='Train Loss', linewidth=2)
        if 'val_loss' in history:
            ax.plot(epochs, history['val_loss'], 'r--', label='Val Loss', linewidth=2)
        
        ax.set_xlabel('Epoch', fontsize=11)
        ax.set_ylabel('Loss', fontsize=11)
        ax.set_title(f"{exp['config']['probe_type']}", fontsize=12, fontweight='bold')
        ax.legend(loc='best')
        ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✓ Training curves plotted successfully!")

## 6. Accuracy Comparison

Compare validation accuracy across epochs for both probe types to see which converges faster and achieves better performance.

In [ ]:
# Plot accuracy comparison
fig, ax = plt.subplots(figsize=(10, 6))

for exp in experiments:
    if 'training_history' in exp:
        history = exp['training_history']
        if 'val_top_1_accuracy' in history:
            epochs = range(1, len(history['val_top_1_accuracy']) + 1)
            ax.plot(epochs, history['val_top_1_accuracy'], 
                   marker='o', linewidth=2.5, markersize=6,
                   label=exp['config']['probe_type'])

ax.set_xlabel('Epoch', fontsize=12)
ax.set_ylabel('Top-1 Accuracy', fontsize=12)
ax.set_title('Validation Accuracy Comparison', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_ylim([0, 1])

plt.tight_layout()
plt.show()

print("\n✓ Accuracy comparison plotted!")

## 7. Show Baseline Comparison

Display final metrics side-by-side to compare probe performance. Hadamard probes typically outperform random probes due to their structured design.

In [ ]:
# Compare final metrics
comparison_data = []
for exp in experiments:
    comparison_data.append({
        'Probe Type': exp['config']['probe_type'],
        'Top-1 Accuracy': exp['metrics'].get('top_1_accuracy', 0),
        'Top-5 Accuracy': exp['metrics'].get('top_5_accuracy', 0),
        'Val Loss': exp['metrics'].get('val_loss', 0),
        'Training Time (s)': exp['training_time_seconds'],
        'Model Params': exp['model_parameters']
    })

comparison_df = pd.DataFrame(comparison_data)

# Format for display
print("\n" + "="*80)
print("PROBE COMPARISON - FINAL METRICS")
print("="*80)
for col in ['Top-1 Accuracy', 'Top-5 Accuracy', 'Val Loss']:
    comparison_df[col] = comparison_df[col].apply(lambda x: f"{x:.4f}")
comparison_df['Training Time (s)'] = comparison_df['Training Time (s)'].apply(lambda x: f"{x:.2f}")

print(comparison_df.to_string(index=False))
print("="*80)

# Determine winner
best_idx = comparison_df['Top-1 Accuracy'].astype(float).idxmax()
best_probe = comparison_data[best_idx]['Probe Type']
print(f"\n🏆 Best Probe: {best_probe}")
print(f"   Top-1 Accuracy: {comparison_data[best_idx]['Top-1 Accuracy']:.4f}")

## Summary and Next Steps

**Congratulations!** You've successfully:
- ✓ Verified your RIS Engine installation
- ✓ Run your first experiment campaign
- ✓ Visualized training curves and accuracy metrics
- ✓ Compared probe performance

### Next Steps:

1. **Interactive Dashboard** → Try `02_dashboard.ipynb` for a full GUI experience with 5 tabs
2. **Run Larger Campaigns** → Explore `03_run_search.ipynb` for comprehensive search spaces
3. **Deep Analysis** → Use `04_analyze_results.ipynb` for statistical analysis and reporting

### Available Campaigns:
- `quick_test.yaml` - Fast validation (2 experiments, ~2 min) ← You just ran this!
- `probe_comparison.yaml` - Compare 6 probe types (18 experiments, ~15 min)
- `model_comparison.yaml` - Compare 3 model architectures (9 experiments, ~10 min)
- `sparsity_sweep.yaml` - Optimize sparsity parameter (45 experiments, ~30 min)
- `full_search.yaml` - Comprehensive search (200+ experiments, several hours)

### Customization:
You can also run individual experiments programmatically:
```python
result = engine.run(
    probe='hadamard',
    model='mlp',
    M=8, K=64, N=64,
    epochs=100
)
engine.show(result)
```

Happy researching! 🚀